<a href="https://colab.research.google.com/github/achiever117/Classification-problems-using-SVM-or-CNN/blob/main/Gans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import Reshape
from keras.datasets.mnist import load_data

In [3]:
def define_disciminator(inp_shape = (28, 28, 1)):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides = (1, 1), padding = 'same', input_shape = inp_shape))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.3))
    model.add(Conv2D(64, (3, 3), strides = (1, 1), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(1, activation = 'sigmoid'))

    opt = Adam(lr = 0.003)
    model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])

    return model


In [4]:
def define_generator(latent_vec):
    latent_vec = (latent_vec, 1)
    gen_model = Sequential()
    nodes = 7*7*64   # output = 3136
    gen_model.add(Flatten())
    gen_model.add(Dense(nodes, input_shape = latent_vec))
    gen_model.add(LeakyReLU(alpha = 0.2))
#     print(gen_model.summary())
    gen_model.add(Reshape((7, 7, 64)))
#     Output --> 7*7*64
    gen_model.add(Conv2DTranspose(32, (4, 4), strides = (2, 2), padding = 'same'))
    gen_model.add(LeakyReLU(alpha = 0.2))
#     Output --> 14*14*32
    gen_model.add(Conv2DTranspose(16, (4, 4), strides = (2, 2), padding = 'same'))
    gen_model.add(LeakyReLU(alpha = 0.2))
#     Output --> 28*28*16
    gen_model.add(Conv2D(1, (3, 3), activation = 'sigmoid', padding = 'same'))    # Output --> 28*28*1

    return gen_model

In [5]:
def define_gan(g_model, d_model):

    d_model.trainable = False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)

    opt = Adam(lr = 0.003)
    model.compile(loss = 'binary_crossentropy', optimizer = opt)

    return model

In [6]:
def load_real_samples():
    (train_X, _), (_, _) = load_data()
    train_X = np.expand_dims(train_X, axis = -1)
    train_X = train_X.astype('float32')
    train_X = train_X/255
    return train_X

In [7]:
def generate_real_samples(dataset, n_samples):
    ids = np.random.randint(0, dataset.shape[0], n_samples)
    dataset = dataset[ids]
    y = np.ones((n_samples, 1))

    return dataset, y

In [8]:
def generate_latent_noise(latent_dim, n_samples):
    noise = np.random.randn(latent_dim * n_samples)
    noise = noise.reshape(n_samples, latent_dim)
    return noise

In [9]:
def generate_fake_samples(g_model, latent_dim, n_samples):
    noise = generate_latent_noise(latent_dim, n_samples)
    fake_data = g_model.predict(noise)
    y = np.zeros((n_samples, 1))

    return fake_data, y

In [10]:
def summary(g_model, d_model, dataset, latent_dim, epoch, n_samples = 30):
    X_real, Y_real = generate_real_samples(dataset, n_samples)

    loss_real, acc_real = d_model.evaluate(X_real, Y_real)

    X_fake, Y_fake = generate_fake_samples(g_model, latent_dim, n_samples)

    loss_fake, acc_fake = d_model.evaluate(X_fake, Y_fake)

    print(f'Accuracy real: {acc_real*100}%, Fake: {acc_fake*100}%')

In [11]:
def train(g_model, d_model, define_gan, dataset, latent_dim, n_epochs = 500, n_batch = 500):
    batch_per_epoch = int(dataset.shape[0]/n_batch)
    n_samples = 250
    for i in range(n_epochs):
        for j in range(batch_per_epoch):
            X_real, Y_real = generate_real_samples(dataset, n_samples)

            X_fake, Y_fake = generate_fake_samples(g_model, latent_dim, n_samples)

            d_loss_real, _ = d_model.train_on_batch(X_real, Y_real)
            d_loss_fake, _ = d_model.train_on_batch(X_fake, Y_fake)         # Till here my generator is frozen

            d_loss = d_loss_fake + d_loss_real                              # Till here discriminator is trained

            noise = generate_latent_noise(latent_dim, n_samples)

            y_gan = np.ones((n_samples, 1))

            g_loss = define_gan.train_on_batch(noise, y_gan)

            print(f'On epoch: {i + 1}, Disc loss: {d_loss}, Gen loss: {g_loss}')

            if i % 10 == 0:
                summary(g_model, d_model, dataset, latent_dim, i)

In [ ]:
latent_dim = 120

d_model = define_disciminator()

g_model = define_generator(latent_dim)

gan = define_gan(g_model, d_model)

dataset = load_real_samples()

train(g_model, d_model, gan, dataset, latent_dim)

8/8 [==============================] - 2s 23ms/step
On epoch: 1, Disc loss: 2.3343928456306458, Gen loss: 0.6138520240783691
1/1 [==============================] - 0s 44ms/step - loss: 0.7482 - accuracy: 0.0000e+00
Accuracy real: 100.0%, Fake: 0.0%
8/8 [==============================] - 0s 12ms/step
On epoch: 1, Disc loss: 1.0653610229492188, Gen loss: 1.2801016569137573
1/1 [==============================] - 0s 49ms/step - loss: 0.3524 - accuracy: 1.0000
Accuracy real: 100.0%, Fake: 100.0%
8/8 [==============================] - 0s 12ms/step
On epoch: 1, Disc loss: 0.859183594584465, Gen loss: 1.8724627494812012
1/1 [==============================] - 0s 73ms/step - loss: 0.2058 - accuracy: 1.0000
Accuracy real: 73.33333492279053%, Fake: 100.0%
8/8 [==============================] - 0s 19ms/step
On epoch: 1, Disc loss: 0.7667819857597351, Gen loss: 2.4293324947357178
1/1 [==============================] - 0s 47ms/step - loss: 0.1233 - accuracy: 1.0000
Accuracy real: 83.33333134651184%, 